# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala _K_-Modes

In [ ]:
import scala.collection.mutable
import smile.plot._
import clustering4ever.scala.clustering.kmodes.KModes
import clustering4ever.math.distances.binary.{Hamming, Vari, MeanMahanttan}
import clustering4ever.scala.clusterizables.{SimpleBinaryClusterizable, BinaryClusterizable}
import clustering4ever.scala.vectorizables.BinaryVector
import clustering4ever.util.ScalaImplicits._
import scala.io.Source
import smile.feature.Standardizer
import smile.data.NumericAttribute
import smile.data.Attribute.Type._
import smile.plot._
import smile.projection._
import smile.manifold._
import java.awt.Color
import clustering4ever.scala.vectorizables.BinaryVectorizable

import scala.collection.mutable
import smile.plot._
import clustering4ever.scala.clustering.kmodes.KModes
import clustering4ever.math.distances.binary.{Hamming, Vari, MeanMahanttan}
import clustering4ever.scala.clusterizables.{SimpleBinaryClusterizable, BinaryClusterizable}
import clustering4ever.scala.vectorizables.BinaryVector
import clustering4ever.util.ScalaImplicits._
import scala.io.Source
import smile.feature.Standardizer
import smile.data.NumericAttribute
import smile.data.Attribute.Type._
import smile.plot._
import smile.projection._
import smile.manifold._
import java.awt.Color
import clustering4ever.scala.vectorizables.BinaryVectorizable


## Download dataset Digits

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Digits/digits.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Digits/labels

--2018-10-27 20:30:09--  http://www.clustering4ever.org/Datasets/Digits/digits.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 959999 (937K) [text/csv]
Saving to: ‘/tmp/digits.csv’

     0K .......... .......... .......... .......... ..........  5% 36.6K 24s
    50K .......... .......... .......... .......... .......... 10% 48.5K 20s
   100K .......... .......... .......... .......... .......... 16%  136K 15s
   150K .......... .......... .......... .......... .......... 21% 13.9K 23s
   200K .......... .......... .......... .......... .......... 26% 45.4K 21s
   250K .......... .......... .......... .......... .......... 32% 93.8K 17s
   300K .......... .......... .......... .......... .......... 37%  225K 14s
   350K .......... .......... .......... .......... .......... 42%  196K 11s
   400K .......... .

## Import and format data with a custom class

In [ ]:
import clustering4ever.scala.clusterizables.BinaryClusterizable
import clustering4ever.scala.vectorizables.BinaryVectorizable
import scala.reflect.ClassTag

// class BinaryClusterizableCustom[ID: Numeric : ClassTag, O : ClassTag, V <: Seq[Int] : ClassTag](idTmp: ID, override val vectorizable: BinaryVectorizable[O, V]) extends 
//   BinaryClusterizable[ID, O, V, BinaryClusterizableCustom[ID, O, V]](idTmp, vectorizable) {
  
//     def setClusterID(newCID: Int): BinaryClusterizableCustom[ID,O,V] = ...
    
//     def setV2(newV2: V): BinaryClusterizableCustom[ID,O,V] = ...    
// }

<console>:26: error: class BinaryClusterizableCustom needs to be abstract, since:
it has 2 unimplemented members.
/** As seen from class BinaryClusterizableCustom, the missing signatures are as follows.
 *  For convenience, these are usable as stub implementations.
 */
  def setClusterID(newCID: Int): BinaryClusterizableCustom[ID,O,V] = ???
  def setV2(newV2: V): BinaryClusterizableCustom[ID,O,V] = ???

       class BinaryClusterizableCustom[ID: Numeric : ClassTag, O : ClassTag, V <: Seq[Int] : ClassTag](idTmp: ID, override val vectorizable: BinaryVectorizable[O, V]) extends
             ^


In [ ]:
val datasetSize = 5000000
val dim = 10
val path = "/tmp/digits.csv"
val useDigitsDS = true

val rawData = if( useDigitsDS ) Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toInt):_*) ).toBuffer
  else mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextInt(2)))

val parData = rawData.zipWithIndex.par.map{ case (v, id) => new SimpleBinaryClusterizable(id.toLong, new BinaryVector(v)) }

val groundTrueLabels = "/tmp/labels"

datasetSize: Int = 5000000
dim: Int = 10
path: String = /tmp/digits.csv
useDigitsDS: Boolean = true
rawData: scala.collection.mutable.Buffer[scala.collection.mutable.ArrayBuffer[Int]] = ArrayBuffer(ArrayBuffer(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,...

## Parameters 

In [ ]:
val k = 10
val iterMax = 100
val epsilon = 0.1
val metric1 = new Hamming[mutable.ArrayBuffer[Int]]

k: Int = 10
iterMax: Int = 100
epsilon: Double = 0.1
metric1: clustering4ever.math.distances.binary.Hamming[scala.collection.mutable.ArrayBuffer[Int]] = clustering4ever.math.distances.binary.Hamming@12e91f9a


## Run the algorithm

In [ ]:
val t1 = System.nanoTime

val model = KModes.run(parData, k, epsilon, iterMax, metric1)

val t2 = System.nanoTime

(t2 - t1) / 1000000000D

t1: Long = 5536481210482
model: clustering4ever.scala.clustering.kmodes.KModesModel[Long,scala.collection.mutable.ArrayBuffer[Int],scala.collection.mutable.ArrayBuffer[Int],clustering4ever.scala.clusterizables.SimpleBinaryClusterizable[Long,scala.collection.mutable.ArrayBuffer[Int],scala.collection.mutable.ArrayBuffer[Int]],clustering4ever.math.distances.binary.Hamming[scala.collection.mutable.ArrayBuffer[Int]]] = clustering4ever.scala.clustering.kmodes.KModesModel@f7d5338
t2: Long = 5536706206724
res13: Double = 0.224996242


0.224996242

## Different ways to apply model to datasets

In [ ]:
val clusterized11 = rawData.map( v => (model.centerPredict(v), v) )
val clusterized12 = model.centerPredict(rawData)

val clusterized21 = parData.map( clustbl => (model.centerPredict(clustbl.vector), clustbl) )
val clusterized22 = model.centerPredict(parData)

clusterized11: scala.collection.mutable.Buffer[(model.ClusterID, scala.collection.mutable.ArrayBuffer[Int])] = ArrayBuffer((7,ArrayBuffer(0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...

### Transform binary data into a grid to visualize it. Here a 15 x 16 grid for digits data

In [ ]:
val centroids = model.centers.toArray
val formatedCentroids = centroids.map(_._2.map(_.toDouble).grouped(15).map(_.toArray).toArray)

centroids: Array[(Int, scala.collection.mutable.ArrayBuffer[Int])] = Array((8,ArrayBuffer(0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,...

### Visualize centroids

In [ ]:
formatedCentroids.foreach( centroid => hexmap(centroid, Array(Palette.BLACK, Palette.LIGHT_GRAY)))

<p align="center">
<img src ="http://www.clustering4ever.org/SparkNotebooksPictures/digits-centers-1.png" width="300" />
<img src ="http://www.clustering4ever.org/SparkNotebooksPictures/digits-centers-2.png" width="300" />
<img src ="http://www.clustering4ever.org/SparkNotebooksPictures/digits-centers-3.png" width="300" />
<img src ="http://www.clustering4ever.org/SparkNotebooksPictures/digits-centers-4.png" width="160" />
</p>

## Standardize data and apply a PCA on it

In [ ]:
val types = for( i <- (0 until parData.head.vector.size).toArray ) yield new NumericAttribute(i.toString)

val standardize = new Standardizer

val dataAsDouble = parData.map(_.vector.map(_.toDouble))

standardize.learn(types.toArray, dataAsDouble.map(_.toArray).toArray)

val standardizedData = clusterized22.map( cz => (cz.clusterID.get, cz.vector.map(_.toDouble).toArray) )

types: Array[smile.data.NumericAttribute] = Array(NUMERIC[0], NUMERIC[1], NUMERIC[2], NUMERIC[3], NUMERIC[4], NUMERIC[5], NUMERIC[6], NUMERIC[7], NUMERIC[8], NUMERIC[9], NUMERIC[10], NUMERIC[11], NUMERIC[12], NUMERIC[13], NUMERIC[14], NUMERIC[15], NUMERIC[16], NUMERIC[17], NUMERIC[18], NUMERIC[19], NUMERIC[20], NUMERIC[21], NUMERIC[22], NUMERIC[23], NUMERIC[24], NUMERIC[25], NUMERIC[26], NUMERIC[27], NUMERIC[28], NUMERIC[29], NUMERIC[30], NUMERIC[31], NUMERIC[32], NUMERIC[33], NUMERIC[34], NUMERIC[35], NUMERIC[36], NUMERIC[37], NUMERIC[38], NUMERIC[39], NUMERIC[40], NUMERIC[41], NUMERIC[42], NUMERIC[43], NUMERIC[44], NUMERIC[45], NUMERIC[46], NUMERIC[47], NUMERIC[48], NUMERIC[49], NUMERIC[50], NUMERIC[51], NUMERIC[52], NUMERIC[53], NUMERIC[54], NUMERIC[55], NUMERIC[56], NUMERIC[57], NUM...

In [ ]:
val readyToPca = standardizedData.map(_._2).toArray

val pcaModel = pca(readyToPca)

pcaModel.setProjection(3)

val pcaizedData = standardizedData.map{ case (clusterID, v) => (clusterID, pcaModel.project(v)) }

readyToPca: Array[Array[Double]] = Array(Array(0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...

In [ ]:
val pureData = pcaizedData.map(_._2).toArray
val labels = pcaizedData.map(_._1).toArray
val pcizedCentroids = centroids.map{ case (_, centroid) =>  (Int.MaxValue, pcaModel.project(standardize.transform(centroid.map(_.toDouble).toArray))) }

pureData: Array[Array[Double]] = Array(Array(0.8581670566923537, 4.491446006457944, 1.3492074366872653), Array(0.3812071416844144, 3.2096660209117225, 0.030868635243768594), Array(2.040289609732212, 3.281287381274164, 0.5883708950800963), Array(0.6009966267867405, 3.1202592201212456, 1.039892669726206), Array(2.188639715061769, 3.017027197526276, 0.15057016702522696), Array(0.33402248854777394, -1.0384366071886362, -0.9078965691317149), Array(2.0582940809526598, 4.122284171373905, 1.9544274974383697), Array(1.5809368270391944, 3.7739566822936377, 1.2726865712592756), Array(1.8419422874284388, 4.275665793297685, 1.8050967388646428), Array(1.2835788275741016, 3.3133122571689695, 1.8535870004193489), Array(1.563510656399107, 3.079184695975594, 0.9349226924825416), Array(2.0847380739786225,...

In [ ]:
plot(pureData, labels, '.', Palette.COLORS)

res19: smile.plot.Window = Window(javax.swing.JFrame[frame0,460,37,1000x1000,invalid,layout=java.awt.BorderLayout,title=Smile Plot 11,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,5,25,990x970,invalid,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777673,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,0x0,invalid,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=9,maximumSize=,minimumSize=,preferredSize=])


Window(javax.swing.JFrame[frame0,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 11,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,34,1000x966,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x966,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])

<p align="center">
<img src ="http://www.clustering4ever.org/SparkNotebooksPictures/digits-pca3.png" width="500" />
</p>

In [ ]:
val window = plot(pureData, labels, '.', Palette.COLORS)
window.canvas.points("centroid", pcizedCentroids.map(_._2), 'Q', Color.BLACK)

window: smile.plot.Window = Window(javax.swing.JFrame[frame1,460,37,1000x1000,invalid,layout=java.awt.BorderLayout,title=Smile Plot 12,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,34,1000x966,invalid,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777673,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,0x0,invalid,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=9,maximumSize=,minimumSize=,preferredSize=])
res21: smile.plot.ScatterPlot = smile.plot.ScatterPlot@662282d


smile.plot.ScatterPlot@662282d

## Visualization with t-sne

In [ ]:
val sne = tsne(readyToPca, 3)

sne: smile.manifold.TSNE = smile.manifold.TSNE@571ae4eb


In [ ]:
plot(sne.getCoordinates, labels, '.', Palette.COLORS)

res24: smile.plot.Window = Window(javax.swing.JFrame[frame2,460,37,1000x1000,invalid,layout=java.awt.BorderLayout,title=Smile Plot 13,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,5,25,990x970,invalid,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777673,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,0x0,invalid,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=9,maximumSize=,minimumSize=,preferredSize=])


Window(javax.swing.JFrame[frame2,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 13,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,34,1000x966,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x966,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])

## Include centroids 

In [ ]:
val readyTsneCentroids = centroids.map{ case (_, centroid) =>  (Int.MaxValue, standardize.transform(centroid.map(_.toDouble).toArray)) }

val readyToTsne = readyToPca ++ readyTsneCentroids.map(_._2)

readyTsneCentroids: Array[(Int, Array[Double])] = Array((2147483647,Array(-0.4691973206300246, -0.724769091822159, 0.9947636535218598, 0.7311989859280826, 0.5818236344745712, 0.4731577041265375, 0.4050460242033411, 0.3752743814604061, 0.3838921816267829, 0.45004943939012954, 0.5440847285433487, 0.7135679682293894, -1.0209698667667724, -0.7473734209867955, -0.4818399603409751, -0.6701124716326963, 0.9433525261840586, 0.6889766255545046, 0.5386811852615968, 0.4419949476766309, 0.36391836974993397, 0.30373202063736837, 0.2784733628071156, 0.3076416819747157, 0.3577255499568549, 0.42899948314926056, 0.5502546049722432, 0.7040158872382919, -1.0353755795234765, -0.6646353033940223, -0.7056049015940564, 0.866566690088696, 0.6622910964242819, 0.5502546049722432, 0.4620448429462271, 0.4500494393...

In [ ]:
val sne2 = tsne(readyToTsne, 3)

sne2: smile.manifold.TSNE = smile.manifold.TSNE@76a1356e


In [ ]:
val dataTsne = sne2.getCoordinates.take(parData.size)
val dataCentroids = sne2.getCoordinates.takeRight(10)

val window = plot(dataTsne, labels, '.', Palette.COLORS)
window.canvas.points("centroid", dataCentroids, 'Q', Color.BLACK)

dataTsne: Array[Array[Double]] = Array(Array(19.92216062072959, 19.412005136547094, -7.175012774070433), Array(22.097242482580935, 16.706418101312586, -2.439049978426416), Array(14.395494347836282, 19.67075210716432, -7.797426170260576), Array(16.296877835278764, 15.609275194936066, -6.873505871551086), Array(18.645759936282843, 16.737739477083082, -0.10628704369422831), Array(17.558370728892303, 3.832026027198662, 7.531168725287251), Array(16.93319735314301, 22.063146455829724, -4.260858500758712), Array(21.20866051579583, 18.210084879763414, -4.098606743270592), Array(19.368510468675478, 22.034118570730374, -5.220287866299489), Array(15.087276301281648, 20.50962073593671, -6.1918922998897985), Array(16.731962071207416, 20.13325035439915, -1.6920538417064932), Array(14.347332591896231,...

smile.plot.ScatterPlot@6b80be31

<p align="center">
<img src ="http://www.clustering4ever.org/SparkNotebooksPictures/digits-tsne3.png" width="500" />
</p>